In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
isamp = 3500
t = np.arange(0,1,0.01)

data_train = spi.loadmat('GP_train_data.mat')

u_in = data_train['u_in']
x_t_in = data_train['x_t_in']
s_in = data_train['s_in']

u_in.shape, x_t_in.shape, s_in.shape

((70000, 100), (70000, 1), (70000, 1))

In [3]:
max_u = np.max(u_in)
min_u = np.min(u_in)

u_in = (u_in-min_u)/(max_u-min_u)

max_t = np.max(x_t_in)
min_t = np.min(x_t_in)

x_t_in = (x_t_in-min_t)/(max_t-min_t)

max_s = np.max(s_in)
min_s = np.min(s_in)

s_in = (s_in-min_s)/(max_s-min_s)

max_s, min_s, max_u, min_u, max_t, min_t

(1.2198099329889192,
 -1.3115838274707885,
 4.208713652908879,
 -3.541514625432276,
 0.99,
 0.0)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 70000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

hln = 25

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = Dense(hln, activation = "relu")(inputsB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (1,), name = 'inputsT')
hiddenT = Dense(hln, activation = "relu")(inputsT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = Dense(1)(combined)

model = Model(inputs = [inputsB, inputsT], outputs = output)    

model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001), loss = 'mse')
model.summary()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = './ChkPts/S1_CII_VDN_s1_10000/',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

itr = 0
for i in range(0, 40):
    print('Iteration: '+str(itr))
    itr = itr+1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = 1,
              verbose = 1, batch_size = bs, callbacks = [model_checkpoint_callback])    

    itr_ps = 250-1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = itr_ps,
              verbose = 0, batch_size = bs, callbacks = [model_checkpoint_callback])
    itr = itr+itr_ps

print('Total iterations: '+str(itr))

model.load_weights('./ChkPts/S1_CII_VDN_s1_10000/')

model.save_weights('./model/S1_CII_VDN_s1_10000')

# model.load_weights('./model/DenseFlipout_model_TFP_weights_30HL_S1_CII_VDN')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 25)           2525        ['inputsB[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 25)           50          ['inputsT[0][0]']                
                                                                                              

In [5]:
data_train = spi.loadmat('GP_matlab.mat')

t = np.arange(0,1,0.01)
u = data_train['u'][10000:20000,:]
s = data_train['s1'][:,10000:20000].T

pointer = np.random.randint(0,100,10000)

u_in_test = np.zeros([1000000,100])
x_t_in_test = np.zeros([1000000,1])
s_in_test = np.zeros([1000000,1])
for i in range(0,10000):
    for j in range(0,100):
        u_in_test[100*i+j,:] = u[i,:]
        x_t_in_test[100*i+j,:] = t[j]
        s_in_test[100*i+j,:] = s[i,j]
        
print(u_in_test.shape, x_t_in_test.shape, s_in_test.shape)

u_in_test = (u_in_test-min_u)/(max_u-min_u)
x_t_in_test = (x_t_in_test-min_t)/(max_t-min_t)
s_in_test = (s_in_test-min_s)/(max_s-min_s)

pred = model({"inputsB":u_in_test, "inputsT":x_t_in_test})

pred = (pred*(max_s-min_s))+min_s 
s_in_test = (s_in_test*(max_s-min_s))+min_s

print()
print(np.mean((s_in_test-pred)**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-pred)**2)/np.mean((s_in_test)**2))    

(1000000, 100) (1000000, 1) (1000000, 1)

1.4497887e-05
0.0380138393414362
0.00038138443647824094
